In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [0]:
import warnings
warnings.filterwarnings("ignore")

import itertools
# import tensorflow as tf
import pandas as pd
import numpy as np
from datetime import date
import os
import time

# from tqdm.notebook import tqdm
# tqdm.pandas()
from tqdm import tqdm
#%matplotlib inline
from sklearn.metrics import mean_squared_error
from functools import reduce

### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,GRU,Dropout
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import datetime
from dateutil.relativedelta import relativedelta
import holidays
from collections import defaultdict
import statsmodels.api as sm
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 100)

from helper_mp import find_len,state_length_check,data_transform_to_id_level,\
dates_checker,dates_checker_2,padder,mean_absolute_percentage_error,create_dataset,create_lstm,\
fit_model,create_and_train_data_for_hw,create_and_train_data_for_arimax,\
create_and_train_data_for_xgboost,compare_models,choose_model_and_forecast,\
three_months_mape_calc,weighted_mape_calc,efd_feature_generator,merge_EFD_feature_file,\
create_and_train_data_for_lstm_multivariate,records_maintainer,metric_file_generator,\
merge_cardcounts_feature_file,\
metric_file_generator_for_exception_accounts,merge_CMD_feature_file,twelve_months_mape_calc,\
twelve_months_rmse_calc,mean_calc,generate_mobility_feature,merge_weighted_fuel_price_feature_file

In [0]:
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import gc
from joblib import Parallel, delayed

In [0]:
## Main holidays
holidays_me = pd.DataFrame()
for yr in range(2008, 2025, 1):
    me_cal = pd.DataFrame(holidays.US(state='ME', years=yr).items(), columns = ['REV_CALENDAR_DATE', 'HOLIDAY_NAME'])
    me_cal['REV_CALENDAR_DATE'] = pd.to_datetime(me_cal['REV_CALENDAR_DATE'])
    me_cal['HOLIDAY'] = 1
    holidays_me = pd.concat([holidays_me, me_cal])

In [0]:
path_params = eval(dataiku.get_custom_variables()['path_params'])
revenue_date_EFD_by_day_path = path_params["revenue_date_EFD_by_day_path"]
daily_gallons_full_path = path_params["daily_gallons_full_path"]

multivariate_lstm_params = eval(dataiku.get_custom_variables()['multivariate_lstm_params'])
train_period_lstm_model = multivariate_lstm_params["train_period_lstm_model"]
pred_period_lstm_model = multivariate_lstm_params["pred_period_lstm_model"]
num_units_lstm_model = multivariate_lstm_params["num_units_lstm_model"]
optimizer_lstm_model = multivariate_lstm_params["optimizer_lstm_model"]
loss_function_lstm_model = multivariate_lstm_params["loss_function_lstm_model"]
batch_size_lstm_model = multivariate_lstm_params["batch_size_lstm_model"]
num_epochs_lstm_model = multivariate_lstm_params["num_epochs_lstm_model"]

hw_params = eval(dataiku.get_custom_variables()['hw_params'])
seasonal_period = hw_params["seasonal_period"]
trend = hw_params["trend"]
seasonal_hw = hw_params["seasonal"]
alpha = hw_params["alpha"]
beta = hw_params["beta"]
gamma = hw_params["gamma"]

arimax_params = eval(dataiku.get_custom_variables()['arimax_params'])
information_criterion = arimax_params["information_criterion"]
seasonal_arimax = arimax_params["seasonal"]
n_jobs = arimax_params["n_jobs"]

xgboost_params = eval(dataiku.get_custom_variables()['xgboost_params'])
learning_rate_xgb = xgboost_params["learning_rate"]
n_estimators_xgb = xgboost_params["n_estimators"]
subsample_xgb = xgboost_params["subsample"]
max_depth_xgb = xgboost_params["max_depth"]
colsample_bytree_xgb = xgboost_params["colsample_bytree"]
min_child_weight_xgb = xgboost_params["min_child_weight"]

primary_params = eval(dataiku.get_custom_variables()['primary_params'])
date_year = primary_params["date_year"]
date_month = primary_params["date_month"]
forecast_target_column = primary_params["forecast_target_column"]
business_program_name = primary_params["business_program_name"]

global_params = eval(dataiku.get_custom_variables()['global_params'])
models = global_params["models"]
model_for_exception_accounts = global_params["model_for_exception_accounts"]
granularity_level = global_params["granularity_level"]
current_month = global_params["current_month"]
validation_window_length = global_params["evaluation_window_length"]
future_prediction_months = global_params["future_prediction_months"]
performance_assessment_window = global_params["performance_assessment_window"]
external_feature_list = global_params["external_feature_list"]
choice = global_params["choice"]

efd_lstm_params = eval(dataiku.get_custom_variables()['efd_lstm_params'])
lstm_train_window_length_EFD = efd_lstm_params["lstm_train_window_length"]
num_units_EFD = efd_lstm_params["num_units"]
activation_function_EFD = efd_lstm_params["activation_function"]
optimizer_EFD = efd_lstm_params["optimizer"]
loss_function_EFD = efd_lstm_params["loss_function"]
batch_size_EFD = efd_lstm_params["batch_size"]
num_epochs_EFD = efd_lstm_params["num_epochs"]
MODELS_EFD = efd_lstm_params["MODELS"]
ma_mapping_EFD = efd_lstm_params["ma_mapping"]

In [0]:
dataset_pd = dataiku.Dataset("volume_gallons_by_account_sanitized").get_dataframe()

DATE = []
for y, m in zip(dataset_pd[date_year], dataset_pd[date_month]):
    DATE.append(date(y, m, 1))
dataset_pd['revenue_date'] = DATE

## data pre processing
dataset_pd['revenue_date'] = pd.to_datetime(dataset_pd['revenue_date'])
dataset_pd.sort_values(['revenue_date'], inplace=True)

dataset_pd['location_state'] = dataset_pd['location_state'].str.upper()

padd1A_states = ['CT','ME','MA','NH','RI','VT']
padd1B_states = ['DE','DC','MD','NJ','NY','PA']
padd1C_states = ['FL','GA','NC','SC','VA','WV']
padd2_states = ['IL','IN','IA','KS','KY','MI','MN','MO','NE','ND','OH','OK','SD','TN','WI']
padd3_states = ['AL','AR','LA','MS','NM','TX']
padd4_states = ['CO','ID','MT','UT','WY']
padd5_states = ['AK','AZ','CA','HI','NV','OR','WA']

conditions = [(dataset_pd['location_state'].isin(padd1A_states)) | (dataset_pd['location_state'].isin(padd1B_states)) | (dataset_pd['location_state'].isin(padd1C_states)) ,
              dataset_pd['location_state'].isin(padd2_states),
              dataset_pd['location_state'].isin(padd3_states),
              dataset_pd['location_state'].isin(padd4_states),
              dataset_pd['location_state'].isin(padd5_states)]
choices =    ['PADD1',
              'PADD2',
              'PADD3',
              'PADD4',
              'PADD5']
dataset_pd['PADD_regions'] = np.select(conditions, choices, default='unknown')

dataset_pd = dataset_pd[dataset_pd['PADD_regions']!='unknown']

dataset_pd.rename(columns = {'global_customer_id':granularity_level},inplace=True)

In [0]:
grpd_by_regions_gallons_sum = dataset_pd.groupby([granularity_level,'revenue_date','PADD_regions']).agg(gallons_sum = ('purchase_gallons_qty',sum))
grpd_by_regions_gallons_sum.reset_index(inplace=True)
grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum.groupby([granularity_level,'revenue_date']).agg(PADD_regions = ('PADD_regions',list),gallons_sum = ('gallons_sum',list))
grpd_by_regions_gallons_sum.reset_index(inplace=True)

In [0]:
mdm = dataiku.Dataset("SANITIZEDATAFOROFFSHORE.mdm_final_sanitized").get_dataframe(columns=['accountnumber','businessprogramname',granularity_level,'location_state'])
#Fill state
id_to_state = dict(zip(mdm[granularity_level],mdm['location_state']))

## Writing code to project gallons at padd level

In [0]:
grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum[(pd.to_datetime(grpd_by_regions_gallons_sum['revenue_date'])<pd.to_datetime(pd.to_datetime(current_month)+pd.DateOffset(months=-(validation_window_length-1))))]

In [0]:
grpd_by_regions_gallons_sum_exploded = grpd_by_regions_gallons_sum.apply(pd.Series.explode)

grpd_by_regions_gallons_sum_required = grpd_by_regions_gallons_sum_exploded.groupby([granularity_level,'PADD_regions']).agg(rev_date = ('revenue_date',list),gallons_list = ('gallons_sum',list))
grpd_by_regions_gallons_sum_required.reset_index(inplace=True)
grpd_by_regions_gallons_sum_required['total_records_before_padding'] = grpd_by_regions_gallons_sum_required.apply(find_len,axis=1)

In [0]:
current_month_efd = str(pd.to_datetime(current_month) + pd.DateOffset(months=-validation_window_length)).split()[0]
validation_window_length_efd = validation_window_length*2
external_feature_list_PADD = ['lag_12','lag_9','lag_6','lag_3','lag_2','lag_1','MA12','MA6','MSD12','MSD6','REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m','covid_mobility_feature']

In [0]:
efd_features_monthly = efd_feature_generator(holidays_me,current_month_efd,lstm_train_window_length_EFD,num_units_EFD,activation_function_EFD,optimizer_EFD,loss_function_EFD,batch_size_EFD,num_epochs_EFD,MODELS_EFD,ma_mapping_EFD,revenue_date_EFD_by_day_path,daily_gallons_full_path,validation_window_length_efd,future_prediction_months)

efd_features_monthly=efd_features_monthly.reset_index()

efd_features_monthly = efd_features_monthly.rename(columns = {'index':'rev_date'})

efd_features_monthly['rev_date'] = pd.to_datetime(efd_features_monthly['rev_date'])

In [0]:
covid_mobility_df = generate_mobility_feature(current_month = current_month_efd, future_prediction_months = validation_window_length_efd)

covid_mobility_df = covid_mobility_df.rename(columns={'month':'rev_date'})
covid_mobility_df = covid_mobility_df.groupby('rev_date').agg(retail_and_recreation_percent_change_from_baseline = ('retail_and_recreation_percent_change_from_baseline',sum))
covid_mobility_df.reset_index(inplace=True)
covid_mobility_df = covid_mobility_df[covid_mobility_df['rev_date']<=(pd.to_datetime(current_month)+ pd.DateOffset(months = validation_window_length))]

In [0]:
def processing_engine_for_gallons_forecast(dataset_pd,grpd_by_regions_gallons_sum_required,current_month_efd,validation_window_length_efd,external_feature_list_PADD):
    start_time = time.time()

    # Import and transform data to the desired level (ID-month level or business_program_name-month level)
    grouped_id_level = grpd_by_regions_gallons_sum_required.copy()
    print(grouped_id_level.shape)

    def dates_checker_gallons(df_dt,current_month):#function designed due to lack of historical data before 2019,else use date_checker2
        row = df_dt['rev_date']
        years_list = set(list(map(lambda x:x.year,row)))
        first_date = row[0]
        last_date = row[-1]
        if first_date <= (current_month + pd.DateOffset(months=-24+1)) and current_month.year -1 in(years_list) and current_month.year in(years_list): #and last_date > pd.to_datetime(current_month):
            return 1
        elif first_date > (current_month + pd.DateOffset(months=-24+1)) and current_month.year in(years_list): #and last_date > pd.to_datetime(current_month):
            return 2
        else:
            return 0

    # Account should have atleast 2 years of data to train check
    grouped_id_level['year_check']  = grouped_id_level.apply(dates_checker_gallons,current_month=pd.to_datetime(current_month_efd),axis=1)
    grouped_id_level = grouped_id_level[grouped_id_level['year_check']==1]
    print(grouped_id_level.shape)
#     grouped_id_level=grouped_id_level.head(30)

    # Missing Value treatment
    combined_date_list = []
    combined_gallons_list = []
    ans = grouped_id_level.apply(padder,
                                  combined_date_list=combined_date_list,
                                  combined_gallons_list=combined_gallons_list,
                                  validation_window_length=validation_window_length,
                                  current_month=pd.to_datetime(current_month),axis=1)
    grouped_id_level['rev_date'] = combined_date_list
    grouped_id_level['gallons_list'] = combined_gallons_list
    grouped_id_level['total_records_after_padding'] = grouped_id_level.apply(find_len,axis=1)
    grouped_id_level_ind = grouped_id_level.reset_index()

    # Adding EFD as a feature
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd'],\
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m'],\
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m']=\
    zip(*grouped_id_level_ind.apply(merge_EFD_feature_file,
                                     efd_features_monthly=efd_features_monthly,
                                     axis=1))

    # Adding covid mobility as a feature
    grouped_id_level_ind['covid_mobility_feature']=grouped_id_level_ind.apply(merge_CMD_feature_file,
                                                                               cmd_features_monthly=covid_mobility_df,
                                                                               axis=1)

    print("--- %s seconds for data preparation---" % (time.time() - start_time))
    # Building models from the multiple options available
    start_time = time.time()

    models_trained = []

    if 'HW' in models:
        print('Running HW')
        rows = [(grouped_id_level_ind.loc[i],
                 pd.to_datetime(current_month_efd),
                 validation_window_length_efd,
                 alpha,
                 beta,
                 gamma,
                 granularity_level) for i in grouped_id_level_ind.index]
        results_hw_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_hw)(a,b,c,d,e,f,g) for a,b,c,d,e,f,g in rows)

        results_hw = pd.DataFrame(results_hw_list, columns=[granularity_level, 'mape_hw', 'rmse_hw',\
                            'agorithm_used_hw', 'rev_date_hw', 'ground_truth_hw', 'predictions_hw'])
        results_hw['agorithm_used_hw'] = 'HW'
        results_hw[[granularity_level,'PADD_regions']]=results_hw[granularity_level].str.split('_',expand=True)
        results_hw[granularity_level] = results_hw[granularity_level].astype('int64')
        models_trained.append(results_hw)
        del rows
        gc.collect()

    if 'XGB' in models and len(external_feature_list)>0:
        print('Running XGB')
        rows = [(grouped_id_level_ind.loc[i],
                pd.to_datetime(current_month_efd),
                validation_window_length_efd,
                external_feature_list_PADD,
                learning_rate_xgb,
                n_estimators_xgb,
                subsample_xgb,
                max_depth_xgb,
                colsample_bytree_xgb,
                min_child_weight_xgb,
                granularity_level) for i in grouped_id_level_ind.index]
        results_xgb_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_xgboost)(a,b,c,d,e,f,g,h,i,j,k) for a,b,c,d,e,f,g,h,i,j,k in rows)

        results_xgb = pd.DataFrame(results_xgb_list, columns=[granularity_level, 'mape_xgb', 'rmse_xgb',
                            'agorithm_used_xgb', 'rev_date_xgb', 'ground_truth_xgb', 'predictions_xgb'])
        results_xgb['agorithm_used_xgb'] = 'XGB'
        results_xgb[[granularity_level,'PADD_regions']]=results_xgb[granularity_level].str.split('_',expand=True)
        results_xgb[granularity_level] = results_xgb[granularity_level].astype('int64')
        models_trained.append(results_xgb)
        del rows
        gc.collect()
    if 'ARIMAX' in models and len(external_feature_list)>0:
        print('Running ARIMAX')
        rows = [(grouped_id_level_ind.loc[i],
                 pd.to_datetime(current_month_efd),
                 validation_window_length_efd,
                 information_criterion,
                 seasonal_arimax,
                 external_feature_list_PADD,
                 n_jobs,
                 granularity_level) for i in grouped_id_level_ind.index]
        results_arimax_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_arimax)(a,b,c,d,e,f,g,h) for a,b,c,d,e,f,g,h in rows)
        results_arimax = pd.DataFrame(results_arimax_list, columns=[
            granularity_level, 'mape_arimax', 'rmse_arimax', 'agorithm_used_arimax',
            'rev_date_arimax', 'ground_truth_arimax', 'predictions_arimax'
        ])
        results_arimax['agorithm_used_arimax'] = 'ARIMAX'
        results_arimax[[granularity_level,'PADD_regions']]=results_arimax[granularity_level].str.split('_',expand=True)
        results_arimax[granularity_level] = results_arimax[granularity_level].astype('int64')
        models_trained.append(results_arimax)
        del rows
        gc.collect()


    if 'LSTM_Multivariate' in models and len(external_feature_list)>0:
        print('Running LSTM_Multivariate')
        rows = [(grouped_id_level_ind.loc[i],
                 pd.to_datetime(current_month_efd),
                 validation_window_length_efd,
                 external_feature_list_PADD,
                 train_period_lstm_model,
                 pred_period_lstm_model,
                 optimizer_lstm_model,
                 loss_function_lstm_model,
                 batch_size_lstm_model,
                 num_epochs_lstm_model,
                granularity_level) for i in grouped_id_level_ind.index]
        results_lstm_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_lstm_multivariate)(a,b,c,d,e,f,g,h,i,j,k) for a,b,c,d,e,f,g,h,i,j,k in rows)

        results_lstm = pd.DataFrame(results_lstm_list, columns=[
            granularity_level, 'mape_lstm_multivariate', 'rmse_lstm_multivariate',
            'agorithm_used_lstm_multivariate',
            'rev_date_lstm_multivariate', 'ground_truth_lstm_multivariate',
            'predictions_lstm_multivariate'
        ])
        results_lstm['agorithm_used_lstm_multivariate'] = 'LSTM_Multivariate'
        results_lstm[[granularity_level,'PADD_regions']]=results_lstm[granularity_level].str.split('_',expand=True)
        results_lstm[granularity_level] = results_lstm[granularity_level].astype('int64')

        models_trained.append(results_lstm)
        del rows
        gc.collect()


    print("--- %s seconds for building models---" % (time.time() - start_time))

    ## COMPARE MODELS
    print('Initialising Compare models')
    start_time = time.time()
    id_best_algo_df = compare_models(models_trained,granularity_level)
    grouped_id_level_ind = grouped_id_level_ind.merge(id_best_algo_df,on=granularity_level,how='left')
    print("--- %s seconds for comparing models---" % (time.time() - start_time))

    ## CHOOSE and FORECAST using the best model
    print('Initialising Forecasting ')
    start_time = time.time()

    rows = [(grouped_id_level_ind.loc[i],
            pd.to_datetime(current_month_efd),
            validation_window_length_efd,
            information_criterion,
            seasonal_arimax,
            external_feature_list_PADD,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind.index]
    final_res_list = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res = pd.DataFrame(final_res_list, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    final_res[[granularity_level,'PADD_regions']]=final_res[granularity_level].str.split('_',expand=True)
    final_res[granularity_level] = final_res[granularity_level].astype('int64')
    del rows
    gc.collect()
    error_file = final_res[final_res['model_chosen']=='ERROR']
    final_res = final_res[final_res['model_chosen']!='ERROR']
    final_res.reset_index(drop=True,inplace=True)

    #final_res['wex_id'],final_res['performance_assesment_mape'],final_res['performance_assesment_rmse'],final_res['agorithm_used'],final_res['performance_assesment_rev_date'],final_res['performance_assesment_ground_truth'],final_res['performance_assesment_forecasts'],final_res['24_months_forecasts']= zip(*grouped_wex_id_level_ind.apply(choose_model_and_forecast,future_prediction_months=future_prediction_months,current_month=current_month,performance_assessment_window=performance_assessment_window,trend=trend,seasonal_hw=seasonal_hw,seasonal_periods=seasonal_period,information_criterion=information_criterion,seasonal_arimax=seasonal_arimax,external_feature_list=external_feature_list,n_jobs=n_jobs,learning_rate_xgb=learning_rate_xgb,n_estimators_xgb=n_estimators_xgb,subsample_xgb=subsample_xgb,max_depth_xgb=max_depth_xgb,colsample_bytree_xgb=colsample_bytree_xgb,min_child_weight_xgb=min_child_weight_xgb,axis=1))
    print("--- %s seconds for forecasting using best model---" % (time.time() - start_time))

    ## Output the required files
    start_time = time.time()
    final_res['3_month_mape(performance_assesment_period)'] = final_res.apply(three_months_mape_calc,axis=1)
    final_res['time_weighted_mape(performance_assesment_period)'] = final_res.apply(weighted_mape_calc,axis=1)

    final_res['overall_mape(performance_assessment_period)'] = final_res['overall_mape(performance_assessment_period)'].astype('float64')
    final_res['time_weighted_mape(performance_assesment_period)'] = final_res['time_weighted_mape(performance_assesment_period)'].astype('float64')
    final_res['3_month_mape(performance_assesment_period)'] = final_res['3_month_mape(performance_assesment_period)'].astype('float64')

    final_res['overall_mape(performance_assessment_period)'] = round(final_res['overall_mape(performance_assessment_period)'],2)
    final_res['time_weighted_mape(performance_assesment_period)'] = round(final_res['time_weighted_mape(performance_assesment_period)'],2)
    final_res['3_month_mape(performance_assesment_period)'] = round(final_res['3_month_mape(performance_assesment_period)'],2)
    df_dict = dict()
    for idx,dfs in enumerate(models_trained):
        model = dfs.iloc[0,3]
        df_dict[model] = idx

    final_res['overall_mape(evaluation_period)'],final_res['3_month_mape(evaluation_period)'],\
    final_res['time_weighted_mape(evaluation_period)'],final_res['monthly_avg_volume(evaluation_period)'] =\
    zip(*final_res.apply(records_maintainer,
                                  df_dict=df_dict,models_trained=models_trained,
                                  granularity_level=granularity_level,axis=1))

    metrics_file = final_res[[granularity_level,'PADD_regions','model_chosen','monthly_avg_volume(evaluation_period)','overall_mape(evaluation_period)',
                             '3_month_mape(evaluation_period)','time_weighted_mape(evaluation_period)',
                             'overall_mape(performance_assessment_period)','3_month_mape(performance_assesment_period)',
                             'time_weighted_mape(performance_assesment_period)']]


    final_res = final_res.merge(grouped_id_level_ind[[granularity_level,'PADD_regions','rev_date','gallons_list']], on =[granularity_level,'PADD_regions'],how='left')
    metric_file = pd.DataFrame()
    metric_file['data'] = final_res.apply(metric_file_generator,
                                                   df_dict=df_dict, models_trained=models_trained,models=models,
                                                   granularity_level=granularity_level,axis=1)
    total_forecast_file = pd.DataFrame()
    for i in range(len(metric_file)):
        total_forecast_file = total_forecast_file.append(metric_file.iloc[i][0],ignore_index=True)

    ##Exception accounts
    print('Handling Exception accounts')
    final_res_B = final_res[final_res['overall_mape(evaluation_period)']>=40]
    final_res11gt40_wids = list(final_res_B[granularity_level])

    grouped_id_level = grpd_by_regions_gallons_sum_required.copy()

    # Account should have atleast 2 years of data to train check
    grouped_id_level['year_check']  = grouped_id_level.apply(dates_checker_gallons,current_month=pd.to_datetime(current_month_efd),axis=1)

    grouped_id_level_ind_A = grouped_id_level.reset_index()

    grouped_id_level_ind_A['ev_mape_gt40_check']  = np.where((grouped_id_level_ind_A[granularity_level].isin(final_res11gt40_wids)),1,0)
    grouped_id_level = grouped_id_level_ind_A.set_index(granularity_level)

    grouped_id_level_AB = grouped_id_level[(grouped_id_level['year_check']==2) | (grouped_id_level['ev_mape_gt40_check']==1)]

    #Padding
    combined_date_list_AB = []
    combined_gallons_list_AB = []
    ans_AB = grouped_id_level_AB.apply(padder,
                                        combined_date_list=combined_date_list_AB,
                                        combined_gallons_list=combined_gallons_list_AB,
                                        validation_window_length=validation_window_length,
                                        current_month=pd.to_datetime(current_month),axis=1)
    grouped_id_level_AB['rev_date'] = combined_date_list_AB
    grouped_id_level_AB['gallons_list'] = combined_gallons_list_AB
    grouped_id_level_AB['total_records_after_padding'] = grouped_id_level_AB.apply(find_len,axis=1)
    grouped_id_level_ind_AB = grouped_id_level_AB.reset_index()
#     grouped_id_level_ind_AB['state'] = grouped_id_level_ind_AB.apply(state_length_check,axis=1)
    grouped_id_level_ind_AB = grouped_id_level_ind_AB[grouped_id_level_ind_AB['total_records_after_padding']>=18]

    ##storing ids
    smallaccounts_ids = list(grouped_id_level_ind_AB[grouped_id_level_ind_AB['year_check']==2][granularity_level])

    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd'],\
    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m'],\
    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m']=\
    zip(*grouped_id_level_ind_AB.apply(merge_EFD_feature_file,efd_features_monthly=efd_features_monthly,axis=1))

    # Adding covid mobility as a feature
    grouped_id_level_ind_AB['covid_mobility_feature']=grouped_id_level_ind_AB.apply(merge_CMD_feature_file,
                                                                                       cmd_features_monthly=covid_mobility_df,
                                                                                       axis=1)

    ## Modelling for Exception accounts
#     grouped_id_level_ind_AB = grouped_id_level_ind_AB.sample(10)
    #ARIMAX
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.assign(best_algo='ARIMAX')   #model_for_exception_accounts
    rows = [(grouped_id_level_ind_AB.loc[i],
            pd.to_datetime(current_month_efd),
            validation_window_length_efd,
            information_criterion,
            seasonal_arimax,
            external_feature_list_PADD,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind_AB.index]
    final_res_exp_list_1 = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res_exp_1 = pd.DataFrame(final_res_exp_list_1, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    final_res_exp_1[[granularity_level,'PADD_regions']]=final_res_exp_1[granularity_level].str.split('_',expand=True)
    final_res_exp_1[granularity_level] = final_res_exp_1[granularity_level].astype('int64')
    del rows
    gc.collect()

    #LSTM_MULTIVARIATE
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.assign(best_algo='LSTM_MULTIVARIATE')   #model_for_exception_accounts
    train_period_lstm_model_exp = 5
    rows = [(grouped_id_level_ind_AB.loc[i],
            pd.to_datetime(current_month_efd),
            validation_window_length_efd,
            information_criterion,
            seasonal_arimax,
            external_feature_list_PADD,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model_exp,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind_AB.index]
    final_res_exp_list_2 = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res_exp_2 = pd.DataFrame(final_res_exp_list_2, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    final_res_exp_2[[granularity_level,'PADD_regions']]=final_res_exp_2[granularity_level].str.split('_',expand=True)
    final_res_exp_2[granularity_level] = final_res_exp_2[granularity_level].astype('int64')
    del rows
    gc.collect()

    #ENSEMBLER
    final_res_exp = pd.DataFrame()
    gallons_forecasts = []

    for i in range(final_res_exp_2.shape[0]):
        gallons_forecasts.append(mean_calc(final_res_exp_1['gallons_forecasts(performance_assessment_period)'].iloc[i],final_res_exp_2['gallons_forecasts(performance_assessment_period)'].iloc[i]))
    final_res_exp[granularity_level] = final_res_exp_2[granularity_level]
    final_res_exp['gallons_actual(performance_assessment_period)'] = final_res_exp_2['gallons_actual(performance_assessment_period)']
    final_res_exp['gallons_forecasts(performance_assessment_period)'] = gallons_forecasts
    final_res_exp['model_chosen'] = 'ENSEMBLE(LSTM_and_ARIMAX)'
    final_res_exp['month'] = final_res_exp_2['month']
    final_res_exp['overall_mape(performance_assessment_period)'] = final_res_exp.apply(twelve_months_mape_calc,axis=1)
    final_res_exp['overall_rmse(performance_assessment_period)'] = final_res_exp.apply(twelve_months_rmse_calc,axis=1)
    final_res_exp['PADD_regions'] = final_res_exp_2['PADD_regions']

    error_file_exp = final_res_exp[final_res_exp['model_chosen']=='ERROR']
    final_res_exp = final_res_exp[final_res_exp['model_chosen']!='ERROR']
    final_res_exp.reset_index(drop=True,inplace=True)

    final_res_exp['3_month_mape(performance_assesment_period)'] = final_res_exp.apply(three_months_mape_calc,axis=1)
    final_res_exp['time_weighted_mape(performance_assesment_period)'] = final_res_exp.apply(weighted_mape_calc,axis=1)

    final_res_exp = final_res_exp.merge(grouped_id_level_ind_AB[[granularity_level,'PADD_regions','rev_date','gallons_list']],on=[granularity_level,'PADD_regions'],how='left')
    metrics_fileAB = final_res_exp[[granularity_level,'model_chosen','overall_mape(performance_assessment_period)',
                              '3_month_mape(performance_assesment_period)','time_weighted_mape(performance_assesment_period)']]

    metric_fileAB = pd.DataFrame()
    metric_fileAB['data'] = final_res_exp.apply(metric_file_generator_for_exception_accounts,granularity_level=granularity_level,axis=1)
    total_forecast_fileAB = pd.DataFrame()
    for i in range(len(metric_fileAB)):
        total_forecast_fileAB = total_forecast_fileAB.append(metric_fileAB.iloc[i][0],ignore_index=True)

    exception_ids = list(final_res_exp[granularity_level])
    metrics_file = metrics_file[~(metrics_file[granularity_level].isin(exception_ids))]
    total_forecast_file = total_forecast_file[~(total_forecast_file[granularity_level].isin(exception_ids))]
    metrics_file = metrics_file[~(metrics_file[granularity_level].isin(exception_ids))]
    final_res = final_res[~(final_res[granularity_level].isin(exception_ids))]

    total_forecast_file = total_forecast_file.append(total_forecast_fileAB).reset_index(drop=True)
    metrics_file = metrics_file.append(metrics_fileAB).reset_index(drop=True)
    final_res = final_res.append(final_res_exp).reset_index(drop=True)

    ## Putting status  flags to differentiate different accounts
    conditions = [final_res[granularity_level].isin(final_res11gt40_wids),
                  final_res[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE',
                  'accounts with less than 36 datapoints']
    final_res['status_flag'] = np.select(conditions, choices, default='eligible accounts')

    conditions = [total_forecast_file[granularity_level].isin(final_res11gt40_wids),
                  total_forecast_file[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE',
                  'accounts with less than 36 datapoints']
    total_forecast_file['status_flag'] = np.select(conditions, choices, default='eligible accounts')

    conditions = [metrics_file[granularity_level].isin(final_res11gt40_wids),
                  metrics_file[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE',
                  'accounts with less than 36 datapoints']
    metrics_file['status_flag'] = np.select(conditions, choices, default='eligible accounts')


    metrics_file.drop(['overall_mape(performance_assessment_period)',
                       '3_month_mape(performance_assesment_period)',
                       'time_weighted_mape(performance_assesment_period)'],axis=1,inplace=True)


#     metrics_file.to_csv('metrics_file.csv',index=False)
#     total_forecast_file.to_csv('forecasts.csv',index=False)
#     final_res.to_csv('final_results.csv',index=False)
    print("--- %s seconds for outputting the required files---" % (time.time() - start_time))


    return final_res,metrics_file,total_forecast_file

In [0]:
final_res_PADD,metrics_file_PADD,total_forecast_file_PADD = processing_engine_for_gallons_forecast(dataset_pd,grpd_by_regions_gallons_sum_required,current_month_efd,validation_window_length_efd,external_feature_list_PADD)

In [0]:
# grpd_by_regions_gallons_sum_required1 = grpd_by_regions_gallons_sum_required.copy()

In [0]:
# grpd_by_regions_gallons_sum_required = grpd_by_regions_gallons_sum_required1.copy()

In [0]:
grpd_by_regions_gallons_sum_required=grpd_by_regions_gallons_sum_required[grpd_by_regions_gallons_sum_required[granularity_level].isin(final_res_PADD[granularity_level])]


grpd_by_regions_gallons_sum_required = grpd_by_regions_gallons_sum_required.sort_values([granularity_level,'PADD_regions'],ascending=[True,True])

grpd_by_regions_gallons_sum_required.reset_index(drop=True,inplace=True)

In [0]:
final_res_PADD = final_res_PADD.sort_values([granularity_level,'PADD_regions'],ascending=[True,True])

final_res_PADD.reset_index(drop=True,inplace=True)

In [0]:
# grpd_by_regions_gallons_sum_required1 = grpd_by_regions_gallons_sum_required.copy()

In [0]:
# grpd_by_regions_gallons_sum_required = grpd_by_regions_gallons_sum_required[[granularity_level,'PADD_regions','rev_date','gallons_list']].merge(final_res_PADD[['month','gallons_forecasts(performance_assessment_period)','model_chosen']],on=grpd_by_regions_gallons_sum_required.index,how='left')

In [0]:
grpd_by_regions_gallons_sum_required = final_res_PADD[[granularity_level,'PADD_regions','month','gallons_forecasts(performance_assessment_period)','model_chosen' ]]

In [0]:
error_file_PADD = grpd_by_regions_gallons_sum_required[grpd_by_regions_gallons_sum_required['model_chosen'].isin(['ERROR','ERRORS'])]
grpd_by_regions_gallons_sum_required = grpd_by_regions_gallons_sum_required[~(grpd_by_regions_gallons_sum_required['model_chosen'].isin(['ERROR','ERRORS']))]

In [0]:
# grpd_by_regions_gallons_sum_required.drop(['key_0','rev_date','gallons_list'],inplace=True,axis=1)

In [0]:
grpd_by_regions_gallons_sum_required.reset_index(drop=True,inplace=True)

In [0]:
def explode_df_for_PADD_regions(df,granularity_level):

    wid = df[granularity_level]
    padd_regions = df['PADD_regions']

    overall_df = pd.DataFrame()
    overall_df['rev_date'] = df['month']
    overall_df['forecasts'] = df['gallons_forecasts(performance_assessment_period)']
    overall_df = overall_df.apply(pd.Series.explode)
    overall_df[granularity_level] = [wid]*len(overall_df)
    overall_df['PADD_regions'] = [padd_regions]*len(overall_df)
    overall_df = overall_df[[granularity_level,'PADD_regions','rev_date','forecasts']]

    return overall_df

In [0]:
metric_fileAB = pd.DataFrame()
metric_fileAB['data'] = grpd_by_regions_gallons_sum_required.apply(explode_df_for_PADD_regions,granularity_level=granularity_level,axis=1)
grpd_by_regions_gallons_sum_required_exploded = pd.DataFrame()
for i in range(len(metric_fileAB)):
    grpd_by_regions_gallons_sum_required_exploded = grpd_by_regions_gallons_sum_required_exploded.append(metric_fileAB.iloc[i][0],ignore_index=True)

In [0]:
grpd_by_regions_gallons_sum_future = grpd_by_regions_gallons_sum_required_exploded.groupby([granularity_level,'rev_date']).agg(PADD_regions = ('PADD_regions',list),gallons_sum = ('forecasts',list))

In [0]:
grpd_by_regions_gallons_sum_future.reset_index(inplace=True)

In [0]:
grpd_by_regions_gallons_sum_future = grpd_by_regions_gallons_sum_future.rename(columns={'rev_date':'revenue_date'})

In [0]:
grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum[grpd_by_regions_gallons_sum[granularity_level].isin(grpd_by_regions_gallons_sum_future[granularity_level].unique())]

In [0]:
grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum.append(grpd_by_regions_gallons_sum_future)
grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum.sort_values([granularity_level,'revenue_date'],ascending=[True,True])

In [0]:
grpd_by_regions_gallons_sum.reset_index(drop=True,inplace=True)

## EFD Prediction

In [0]:
efd_features_monthly = efd_feature_generator(holidays_me,current_month,lstm_train_window_length_EFD,num_units_EFD,activation_function_EFD,optimizer_EFD,loss_function_EFD,batch_size_EFD,num_epochs_EFD,MODELS_EFD,ma_mapping_EFD,revenue_date_EFD_by_day_path,daily_gallons_full_path,validation_window_length,future_prediction_months)

efd_features_monthly=efd_features_monthly.reset_index()

efd_features_monthly = efd_features_monthly.rename(columns = {'index':'rev_date'})

efd_features_monthly['rev_date'] = pd.to_datetime(efd_features_monthly['rev_date'])

## Google mobility feature for covid months

In [0]:
covid_mobility_df = generate_mobility_feature(current_month = current_month, future_prediction_months = validation_window_length)

covid_mobility_df = covid_mobility_df.rename(columns={'month':'rev_date'})
covid_mobility_df = covid_mobility_df.groupby('rev_date').agg(retail_and_recreation_percent_change_from_baseline = ('retail_and_recreation_percent_change_from_baseline',sum))
covid_mobility_df.reset_index(inplace=True)
covid_mobility_df = covid_mobility_df[covid_mobility_df['rev_date']<=pd.to_datetime(current_month)+ pd.DateOffset(months = validation_window_length)]
# covid_mobility_df.tail()

## Adding fuel prices as feature

In [0]:
# fuel_price_data = pd.read_csv('D:/Users/W505723/Downloads/Sanitized_dataset/fule_prices_data_projections.csv')
fuel_price_data = dataiku.Dataset("fule_prices_data_projections").get_dataframe()

fuel_price_data = fuel_price_data.T

fuel_price_data.columns = [fuel_price_data.iloc[0]]
fuel_price_data = fuel_price_data[1:]
fuel_price_data  = fuel_price_data.iloc[: , :5]
fuel_price_data.reset_index(inplace=True)
fuel_price_data.columns = fuel_price_data.columns.get_level_values(0)
dates = pd.date_range('1997-01-01','12-01-2023',freq='MS')
fuel_price_data['rev_date'] = dates
fuel_price_data.drop(['index'],axis=1,inplace=True)


fuel_price_data = fuel_price_data.rename(columns={'PADD 1 (East Coast)':'PADD_1',\
                                                  'PADD 2 (Midwest)':'PADD_2',\
                                                  'PADD 3 (Gulf Coast)':'PADD_3',\
                                                  'PADD 4 (Rocky Mountain)':'PADD_4',\
                                                  'PADD 5 (West Coast)':'PADD_5'})

fuel_price_data[['PADD_1', 'PADD_2', 'PADD_3', 'PADD_4', 'PADD_5']] = fuel_price_data[['PADD_1', 'PADD_2', 'PADD_3', 'PADD_4', 'PADD_5']]/100

In [0]:
fuel_price_data=fuel_price_data[fuel_price_data['rev_date']<= (pd.to_datetime(current_month)+ pd.DateOffset(months = validation_window_length))]

In [0]:
grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum.merge(fuel_price_data,left_on='revenue_date',right_on='rev_date',how='left')

In [0]:
# grpd_by_regions_gallons_sum1 = grpd_by_regions_gallons_sum.copy()

In [0]:
# grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum1.copy()

In [0]:
# def checkIfDuplicates_1(df):
#     listOfElems = df['PADD_regions']
#     print(listOfElems,len(listOfElems),str(listOfElems),len(str(listOfElems)),'*****')
#     ''' Check if given list contains any duplicates '''
#     if len(listOfElems) == len(set(listOfElems)):
#         return False
#     else:
#         return True
# grpd_by_regions_gallons_sum['dups'] = grpd_by_regions_gallons_sum.apply(checkIfDuplicates_1,axis=1)

In [0]:
# grpd_by_regions_gallons_sum = grpd_by_regions_gallons_sum.tail(12000)

In [0]:
print('Calculating weighted fuel price')
def weighted_average_fuel_price(df):
    try:
        fuel_prices_PADDwise = []
        fuel_prices_PADDwise.append(df['PADD_1'])
        fuel_prices_PADDwise.append(df['PADD_2'])
        fuel_prices_PADDwise.append(df['PADD_3'])
        fuel_prices_PADDwise.append(df['PADD_4'])
        fuel_prices_PADDwise.append(df['PADD_5'])

        padd_regions_ind = df['PADD_regions']
        padd_regions_gallons = df['gallons_sum']
        df2 =  pd.DataFrame({'padd_regions_gallons': padd_regions_gallons},index=padd_regions_ind)
        if df2.index.is_unique==False:
            print(False)
        df2 = df2.groupby(df2.index).sum()
        padd_regions_new_ind = ['PADD1', 'PADD2', 'PADD3', 'PADD4', 'PADD5']
        df2 = df2.reindex(padd_regions_new_ind, fill_value=0)
        weights_PADDwise = df2['padd_regions_gallons']

        weighted_sum = []
        for price, weights in zip(fuel_prices_PADDwise, weights_PADDwise):
            weighted_sum.append(price * weights)

        weighted_fuel_price = round(sum(weighted_sum) / (sum(weights_PADDwise)+0.0001),3)
        weighted_fuel_price_x_gallons = weighted_fuel_price* sum(weights_PADDwise)
        return weighted_fuel_price,weighted_fuel_price_x_gallons
    except:
        print('ERROR in : ',df['global_customer_id'])
        return 0,0


grpd_by_regions_gallons_sum['weighted_average_fuel_price'],grpd_by_regions_gallons_sum['weighted_fuel_price_x_gallons'] = zip(*grpd_by_regions_gallons_sum.apply(weighted_average_fuel_price,axis=1))

In [0]:
weighted_fuel_price_feature = grpd_by_regions_gallons_sum[[granularity_level,'revenue_date','weighted_average_fuel_price','weighted_fuel_price_x_gallons']]
weighted_fuel_price_feature = weighted_fuel_price_feature.rename(columns={'revenue_date':'rev_date'})

In [0]:
weighted_fuel_price_feature = weighted_fuel_price_feature[weighted_fuel_price_feature[granularity_level].isin(grpd_by_regions_gallons_sum_required[granularity_level].unique())]

## Pipeline

In [0]:
# bill_codes_df = pd.read_csv('D:/Users/W505723/WEX_Data_exploration/EDA_NewSanitisedData/Revenue Forecasting/revenue_code_groups.csv')
# rebates_codes = list(bill_codes_df[bill_codes_df['Revenue Code Group'] == 'Rebate']['Revenue Code'].values)
# dataset_pd = dataset_pd[dataset_pd['revenue_code'].isin(rebates_codes)]

dataset_pd = dataset_pd[(dataset_pd['revenue_code']=='M01')]

In [0]:
# def data_transform_to_id_level(ten_yrs_data,forecast_target_column,granularity_level):
#     # grouping at id month level and calculating total_purchase_gallons_quantity/revenue
#     grouped_id_level = ten_yrs_data.groupby([granularity_level,'revenue_date']).agg(total_values = (forecast_target_column,sum))
#     grouped_id_level.reset_index(inplace=True)
#     grouped_id_level = grouped_id_level.groupby(granularity_level).agg(rev_date = ('revenue_date',list),gallons_list = ('total_values',list))

#     grouped_id_level['total_records'] = grouped_id_level.apply(find_len,axis=1)

#     grouped_id_level.sort_values('total_records',ascending=False,inplace = True)

#     return grouped_id_level

In [0]:
def processing_engine(dataset_pd):

    start_time = time.time()
    # Import and transform data to the desired level (WEX_ID-month level or business_program_name-month level)
    grouped_id_level = data_transform_to_id_level(dataset_pd,forecast_target_column,granularity_level)
    # Account should have atleast 2 years of data to train check
    if choice == 1: #For only Forecast
        print(1)
        grouped_id_level['year_check']  = grouped_id_level.apply(dates_checker_2,current_month=pd.to_datetime(current_month),axis=1)
        grouped_id_level = grouped_id_level[grouped_id_level['year_check']==1]
#         grouped_id_level=grouped_id_level.head(5)

    if choice == 2: #For Forecast and Compare
        print(2)
        grouped_id_level['year_check']  = grouped_id_level.apply(dates_checker,current_month=pd.to_datetime(current_month),axis=1)
        grouped_id_level = grouped_id_level[grouped_id_level['year_check']==1]
#         grouped_id_level=grouped_id_level.head(30)

    # Missing Value treatment
    combined_date_list = []
    combined_gallons_list = []
    ans = grouped_id_level.apply(padder,
                                  combined_date_list=combined_date_list,
                                  combined_gallons_list=combined_gallons_list,
                                  validation_window_length=validation_window_length,
                                  current_month=pd.to_datetime(current_month),axis=1)
    grouped_id_level['rev_date'] = combined_date_list
    grouped_id_level['gallons_list'] = combined_gallons_list
    grouped_id_level['total_records_after_padding'] = grouped_id_level.apply(find_len,axis=1)
    grouped_id_level_ind = grouped_id_level.reset_index()

    grouped_id_level_ind = grouped_id_level_ind[grouped_id_level_ind[granularity_level].
                                                              isin(grpd_by_regions_gallons_sum_required[granularity_level].
                                                                   unique())]

    print(grouped_id_level_ind.shape)
    # Adding EFD as a feature
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd'],\
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m'],grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m']=\
    zip(*grouped_id_level_ind.apply(merge_EFD_feature_file,
                                             efd_features_monthly=efd_features_monthly,axis=1))

    # Adding covid mobility as a feature
    grouped_id_level_ind['covid_mobility_feature']=grouped_id_level_ind.apply(merge_CMD_feature_file,
                                                                                       cmd_features_monthly=covid_mobility_df,
                                                                                       axis=1)

    # Adding fuel_price_data as a feature
    grouped_id_level_ind['weighted_average_fuel_price'],grouped_id_level_ind['weighted_fuel_price_x_gallons']=\
    zip(*grouped_id_level_ind.apply(merge_weighted_fuel_price_feature_file,
                                             weighted_fuel_price_feature=weighted_fuel_price_feature,
                                             granularity_level=granularity_level,
                                             axis=1))

    print("--- %s seconds for data preparation---" % (time.time() - start_time))
    # Building models from the multiple options available
    start_time = time.time()
    print(grouped_id_level_ind.shape)
    models_trained = []

    if 'HW' in models:
        start_time_hw = time.time()
        print('Running HW')
        rows = [(grouped_id_level_ind.loc[i],
                 pd.to_datetime(current_month),
                 validation_window_length,
                 alpha,
                 beta,
                 gamma,
                 granularity_level) for i in grouped_id_level_ind.index]
        results_hw_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_hw)(a,b,c,d,e,f,g) for a,b,c,d,e,f,g in rows)

        results_hw = pd.DataFrame(results_hw_list, columns=[granularity_level, 'mape_hw', 'rmse_hw',\
                            'agorithm_used_hw', 'rev_date_hw', 'ground_truth_hw', 'predictions_hw'])
        results_hw['agorithm_used_hw'] = 'HW'
        models_trained.append(results_hw)
        del rows
        gc.collect()
        print("--- %s seconds for running HW---" % (time.time() - start_time_hw))


    if 'ARIMAX' in models and len(external_feature_list)>0:
        start_time_arimax = time.time()
        print('Running ARIMAX')
        rows = [(grouped_id_level_ind.loc[i],
                 pd.to_datetime(current_month),
                 validation_window_length,
                 information_criterion,
                 seasonal_arimax,
                 external_feature_list,
                 n_jobs,
                 granularity_level) for i in grouped_id_level_ind.index]
        results_arimax_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_arimax)(a,b,c,d,e,f,g,h) for a,b,c,d,e,f,g,h in rows)
        results_arimax = pd.DataFrame(results_arimax_list, columns=[
            granularity_level, 'mape_arimax', 'rmse_arimax', 'agorithm_used_arimax',
            'rev_date_arimax', 'ground_truth_arimax', 'predictions_arimax'
        ])
        results_arimax['agorithm_used_arimax'] = 'ARIMAX'
        models_trained.append(results_arimax)
        del rows
        gc.collect()
        print("--- %s seconds for running ARIMAX---" % (time.time() - start_time_arimax))
    if 'XGB' in models and len(external_feature_list)>0:
        start_time_xgb = time.time()
        print('Running XGB')
        rows = [(grouped_id_level_ind.loc[i],
                pd.to_datetime(current_month),
                validation_window_length,
                external_feature_list,
                learning_rate_xgb,
                n_estimators_xgb,
                subsample_xgb,
                max_depth_xgb,
                colsample_bytree_xgb,
                min_child_weight_xgb,
                granularity_level) for i in grouped_id_level_ind.index]
        results_xgb_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_xgboost)(a,b,c,d,e,f,g,h,i,j,k) for a,b,c,d,e,f,g,h,i,j,k in rows)

        results_xgb = pd.DataFrame(results_xgb_list, columns=[granularity_level, 'mape_xgb', 'rmse_xgb',
                            'agorithm_used_xgb', 'rev_date_xgb', 'ground_truth_xgb', 'predictions_xgb'])
        results_xgb['agorithm_used_xgb'] = 'XGB'
        models_trained.append(results_xgb)
        del rows
        gc.collect()
        print("--- %s seconds for running XGB---" % (time.time() - start_time_xgb))

    if 'LSTM_Multivariate' in models and len(external_feature_list)>0:
        start_time_lstm = time.time()
        print('Running LSTM_Multivariate')
        rows = [(grouped_id_level_ind.loc[i],
                 pd.to_datetime(current_month),
                 validation_window_length,
                 external_feature_list,
                 train_period_lstm_model,
                 pred_period_lstm_model,
                 optimizer_lstm_model,
                 loss_function_lstm_model,
                 batch_size_lstm_model,
                 num_epochs_lstm_model,
                granularity_level) for i in grouped_id_level_ind.index]
        results_lstm_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_lstm_multivariate)(a,b,c,d,e,f,g,h,i,j,k) for a,b,c,d,e,f,g,h,i,j,k in rows)

        results_lstm = pd.DataFrame(results_lstm_list, columns=[
            granularity_level, 'mape_lstm_multivariate', 'rmse_lstm_multivariate',
            'agorithm_used_lstm_multivariate',
            'rev_date_lstm_multivariate', 'ground_truth_lstm_multivariate',
            'predictions_lstm_multivariate'
        ])
        results_lstm['agorithm_used_lstm_multivariate'] = 'LSTM_Multivariate'
        models_trained.append(results_lstm)
        del rows
        gc.collect()
        print("--- %s seconds for running LSTM_Multivariate---" % (time.time() - start_time_lstm))

    print("--- %s seconds for building models---" % (time.time() - start_time))

    ## COMPARE MODELS
    print('Initialising Compare models')
    start_time = time.time()
    id_best_algo_df = compare_models(models_trained,granularity_level)
    grouped_id_level_ind = grouped_id_level_ind.merge(id_best_algo_df,on=granularity_level,how='left')
    print("--- %s seconds for comparing models---" % (time.time() - start_time))

    ## CHOOSE and FORECAST using the best model
    print('Initialising Forecasting ')
    start_time = time.time()

    rows = [(grouped_id_level_ind.loc[i],
            pd.to_datetime(current_month),
            validation_window_length,
            information_criterion,
            seasonal_arimax,
            external_feature_list,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind.index]
    final_res_list = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res = pd.DataFrame(final_res_list, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    del rows
    gc.collect()
    error_file = final_res[final_res['model_chosen']=='ERROR']
    final_res = final_res[final_res['model_chosen']!='ERROR']

    #final_res['wex_id'],final_res['performance_assesment_mape'],final_res['performance_assesment_rmse'],final_res['agorithm_used'],final_res['performance_assesment_rev_date'],final_res['performance_assesment_ground_truth'],final_res['performance_assesment_forecasts'],final_res['24_months_forecasts']= zip(*grouped_wex_id_level_ind.apply(choose_model_and_forecast,future_prediction_months=future_prediction_months,current_month=current_month,performance_assessment_window=performance_assessment_window,trend=trend,seasonal_hw=seasonal_hw,seasonal_periods=seasonal_period,information_criterion=information_criterion,seasonal_arimax=seasonal_arimax,external_feature_list=external_feature_list,n_jobs=n_jobs,learning_rate_xgb=learning_rate_xgb,n_estimators_xgb=n_estimators_xgb,subsample_xgb=subsample_xgb,max_depth_xgb=max_depth_xgb,colsample_bytree_xgb=colsample_bytree_xgb,min_child_weight_xgb=min_child_weight_xgb,axis=1))
    print("--- %s seconds for forecasting using best model---" % (time.time() - start_time))

    ## Output the required files
    start_time = time.time()
    final_res['3_month_mape(performance_assesment_period)'] = final_res.apply(three_months_mape_calc,axis=1)
    final_res['time_weighted_mape(performance_assesment_period)'] = final_res.apply(weighted_mape_calc,axis=1)

    final_res['overall_mape(performance_assessment_period)'] = final_res['overall_mape(performance_assessment_period)'].astype('float64')
    final_res['time_weighted_mape(performance_assesment_period)'] = final_res['time_weighted_mape(performance_assesment_period)'].astype('float64')
    final_res['3_month_mape(performance_assesment_period)'] = final_res['3_month_mape(performance_assesment_period)'].astype('float64')

    final_res['overall_mape(performance_assessment_period)'] = round(final_res['overall_mape(performance_assessment_period)'],2)
    final_res['time_weighted_mape(performance_assesment_period)'] = round(final_res['time_weighted_mape(performance_assesment_period)'],2)
    final_res['3_month_mape(performance_assesment_period)'] = round(final_res['3_month_mape(performance_assesment_period)'],2)
    df_dict = dict()
    for idx,dfs in enumerate(models_trained):
        model = dfs.iloc[0,3]
        df_dict[model] = idx

    final_res['overall_mape(evaluation_period)'],final_res['3_month_mape(evaluation_period)'],\
    final_res['time_weighted_mape(evaluation_period)'],final_res['monthly_avg_volume(evaluation_period)'] =\
    zip(*final_res.apply(records_maintainer,
                                  df_dict=df_dict,models_trained=models_trained,
                                  granularity_level=granularity_level,axis=1))

    metrics_file = final_res[[granularity_level,'model_chosen','monthly_avg_volume(evaluation_period)','overall_mape(evaluation_period)',
                             '3_month_mape(evaluation_period)','time_weighted_mape(evaluation_period)',
                             'overall_mape(performance_assessment_period)','3_month_mape(performance_assesment_period)',
                             'time_weighted_mape(performance_assesment_period)']]


    final_res = final_res.merge(grouped_id_level_ind[[granularity_level,'rev_date','gallons_list']], on =granularity_level,how='left')
    metric_file = pd.DataFrame()
    metric_file['data'] = final_res.apply(metric_file_generator,
                                                   df_dict=df_dict, models_trained=models_trained,models=models,
                                                   granularity_level=granularity_level,axis=1)
    total_forecast_file = pd.DataFrame()
    for i in range(len(metric_file)):
        total_forecast_file = total_forecast_file.append(metric_file.iloc[i][0],ignore_index=True)

    ##Exception accounts
    print('Handling Exception accounts')
    final_res_B = final_res[final_res['overall_mape(evaluation_period)']>=40]
    final_res11gt40_wids = list(final_res_B[granularity_level])

    grouped_id_level = data_transform_to_id_level(dataset_pd,forecast_target_column,granularity_level)
    if choice == 1:
        print(1)
        grouped_id_level['year_check']  = grouped_id_level.apply(dates_checker_2,current_month=pd.to_datetime(current_month),axis=1)
    if choice == 2:
        print(2)
        grouped_id_level['year_check']  = grouped_id_level.apply(dates_checker,current_month=pd.to_datetime(current_month),axis=1)

    grouped_id_level_ind_A = grouped_id_level.reset_index()

    grouped_id_level_ind_A['ev_mape_gt40_check']  = np.where((grouped_id_level_ind_A[granularity_level].isin(final_res11gt40_wids)),1,0)
    grouped_id_level = grouped_id_level_ind_A.set_index(granularity_level)

    grouped_id_level_AB = grouped_id_level[(grouped_id_level['year_check']==2) | (grouped_id_level['ev_mape_gt40_check']==1)]

    #Padding
    combined_date_list_AB = []
    combined_gallons_list_AB = []
    ans_AB = grouped_id_level_AB.apply(padder,
                                                combined_date_list=combined_date_list_AB,
                                                combined_gallons_list=combined_gallons_list_AB,
                                                validation_window_length=validation_window_length,
                                                current_month=pd.to_datetime(current_month),axis=1)
    grouped_id_level_AB['rev_date'] = combined_date_list_AB
    grouped_id_level_AB['gallons_list'] = combined_gallons_list_AB
    grouped_id_level_AB['total_records_after_padding'] = grouped_id_level_AB.apply(find_len,axis=1)
    grouped_id_level_ind_AB = grouped_id_level_AB.reset_index()
#     grouped_id_level_ind_AB['state'] = grouped_id_level_ind_AB.apply(state_length_check,axis=1)
    grouped_id_level_ind_AB = grouped_id_level_ind_AB[grouped_id_level_ind_AB['total_records_after_padding']>=18]

    ##storing ids
    smallaccounts_ids = list(grouped_id_level_ind_AB[grouped_id_level_ind_AB['year_check']==2][granularity_level])

    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd'],\
    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m'],\
    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m']=\
    zip(*grouped_id_level_ind_AB.apply(merge_EFD_feature_file,efd_features_monthly=efd_features_monthly,axis=1))

    # Adding covid mobility as a feature
    grouped_id_level_ind_AB['covid_mobility_feature']=grouped_id_level_ind_AB.apply(merge_CMD_feature_file,
                                                                                       cmd_features_monthly=covid_mobility_df,
                                                                                       axis=1)

    # Adding fuel_price_data as a feature
    grouped_id_level_ind_AB['weighted_average_fuel_price'],grouped_id_level_ind_AB['weighted_fuel_price_x_gallons']=\
    zip(*grouped_id_level_ind_AB.apply(merge_weighted_fuel_price_feature_file,
                                             weighted_fuel_price_feature=weighted_fuel_price_feature,
                                             granularity_level=granularity_level,
                                             axis=1))

    ## Modelling for Exception accounts
#     grouped_id_level_ind_AB = grouped_id_level_ind_AB.sample(5)
    #ARIMAX
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.assign(best_algo='ARIMAX')   #model_for_exception_accounts
    rows = [(grouped_id_level_ind_AB.loc[i],
            pd.to_datetime(current_month),
            validation_window_length,
            information_criterion,
            seasonal_arimax,
            external_feature_list,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind_AB.index]
    final_res_exp_list_1 = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res_exp_1 = pd.DataFrame(final_res_exp_list_1, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    del rows
    gc.collect()

    #LSTM_MULTIVARIATE
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.assign(best_algo='LSTM_MULTIVARIATE')   #model_for_exception_accounts
    train_period_lstm_model_exp = 5
    rows = [(grouped_id_level_ind_AB.loc[i],
            pd.to_datetime(current_month),
            validation_window_length,
            information_criterion,
            seasonal_arimax,
            external_feature_list,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model_exp,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind_AB.index]
    final_res_exp_list_2 = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res_exp_2 = pd.DataFrame(final_res_exp_list_2, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    del rows
    gc.collect()

    #ENSEMBLER
    final_res_exp = pd.DataFrame()
    gallons_forecasts = []

    for i in range(final_res_exp_2.shape[0]):
        gallons_forecasts.append(mean_calc(final_res_exp_1['gallons_forecasts(performance_assessment_period)'].iloc[i],final_res_exp_2['gallons_forecasts(performance_assessment_period)'].iloc[i]))
    final_res_exp[granularity_level] = final_res_exp_2[granularity_level]
    final_res_exp['gallons_actual(performance_assessment_period)'] = final_res_exp_2['gallons_actual(performance_assessment_period)']
    final_res_exp['gallons_forecasts(performance_assessment_period)'] = gallons_forecasts
    final_res_exp['model_chosen'] = 'ENSEMBLE(LSTM_and_ARIMAX)'
    final_res_exp['month'] = final_res_exp_2['month']
    final_res_exp['overall_mape(performance_assessment_period)'] = final_res_exp.apply(twelve_months_mape_calc,axis=1)
    final_res_exp['overall_rmse(performance_assessment_period)'] = final_res_exp.apply(twelve_months_rmse_calc,axis=1)

    error_file_exp = final_res_exp[final_res_exp['model_chosen']=='ERROR']
    final_res_exp = final_res_exp[final_res_exp['model_chosen']!='ERROR']

    final_res_exp['3_month_mape(performance_assesment_period)'] = final_res_exp.apply(three_months_mape_calc,axis=1)
    final_res_exp['time_weighted_mape(performance_assesment_period)'] = final_res_exp.apply(weighted_mape_calc,axis=1)

    final_res_exp = final_res_exp.merge(grouped_id_level_ind_AB[[granularity_level,'rev_date','gallons_list']],on=granularity_level,how='left')
    metrics_fileAB = final_res_exp[[granularity_level,'model_chosen','overall_mape(performance_assessment_period)',
                              '3_month_mape(performance_assesment_period)','time_weighted_mape(performance_assesment_period)']]

    metric_fileAB = pd.DataFrame()
    metric_fileAB['data'] = final_res_exp.apply(metric_file_generator_for_exception_accounts,granularity_level=granularity_level,axis=1)
    total_forecast_fileAB = pd.DataFrame()
    for i in range(len(metric_fileAB)):
        total_forecast_fileAB = total_forecast_fileAB.append(metric_fileAB.iloc[i][0],ignore_index=True)

    exception_ids = list(final_res_exp[granularity_level])
    metrics_file = metrics_file[~(metrics_file[granularity_level].isin(exception_ids))]
    total_forecast_file = total_forecast_file[~(total_forecast_file[granularity_level].isin(exception_ids))]
    metrics_file = metrics_file[~(metrics_file[granularity_level].isin(exception_ids))]
    final_res = final_res[~(final_res[granularity_level].isin(exception_ids))]

    total_forecast_file = total_forecast_file.append(total_forecast_fileAB).reset_index(drop=True)
    metrics_file = metrics_file.append(metrics_fileAB).reset_index(drop=True)
    final_res = final_res.append(final_res_exp).reset_index(drop=True)

    ## Putting status  flags to differentiate different accounts
    conditions = [final_res[granularity_level].isin(final_res11gt40_wids),
                  final_res[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE',
                  'accounts with less than 36 datapoints']
    final_res['status_flag'] = np.select(conditions, choices, default='eligible accounts')

    conditions = [total_forecast_file[granularity_level].isin(final_res11gt40_wids),
                  total_forecast_file[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE',
                  'accounts with less than 36 datapoints']
    total_forecast_file['status_flag'] = np.select(conditions, choices, default='eligible accounts')

    conditions = [metrics_file[granularity_level].isin(final_res11gt40_wids),
                  metrics_file[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE',
                  'accounts with less than 36 datapoints']
    metrics_file['status_flag'] = np.select(conditions, choices, default='eligible accounts')

    if choice == 1:
        print(1)
        metrics_file.drop(['overall_mape(performance_assessment_period)',
                           '3_month_mape(performance_assesment_period)',
                           'time_weighted_mape(performance_assesment_period)'],axis=1,inplace=True)

#     metrics_file.to_csv('metrics_file.csv',index=False)
#     total_forecast_file.to_csv('forecasts.csv',index=False)
#     final_res.to_csv('final_results.csv',index=False)

    # Write recipe outputs
    metrics = dataiku.Dataset("metrics")
    metrics.write_with_schema(metrics_file)
    forecasts = dataiku.Dataset("forecasts")
    forecasts.write_with_schema(total_forecast_file)

    print("--- %s seconds for outputting the required files---" % (time.time() - start_time))


    return models_trained,grouped_id_level_ind,id_best_algo_df,final_res,total_forecast_file,metrics_file,error_file,error_file_exp

In [0]:
models_trained,grouped_id_level_ind,id_best_algo_df,final_res,\
total_forecast_file,metrics_file,error_file,error_file_exp= processing_engine(dataset_pd)